In [68]:
from bs4 import BeautifulSoup
import urllib.request
from datetime import date
import pandas as pd
date_today = date.today()

In [70]:
product_url = []
name = []
sales_price = []
price = []
picture_url = []
brand = []
product_type = []
timestamp  = []

for i in range(1,13): #change variable man 14
    ourUrl = urllib.request.urlopen('https://maisonkitsune.com/us/women/ready-to-wear.html?p=' + str(i))
    #ourUrl = urllib.request.urlopen('https://maisonkitsune.com/us/men/ready-to-wear.html?p='+str(i))
    soup = BeautifulSoup(ourUrl,'html.parser')

    for i in soup.find_all('li',{'class':'item product product-item'}):
        product_url.append(i.find('a',{'class':'product-item-link'}).get('href'))
        name.append(i.find('a',{'class':'product-item-link'}).text.strip().lower().capitalize())
        timestamp.append(date_today)
        brand.append("Maison Kitsune")
    
        product_type.append("Women's Clothing")
        if i.find('a',{'class':'product photo product-item-photo'}).source.get('data-srcset') != None:
            picture_url.append(i.find('a',{'class':'product photo product-item-photo'}).source.get('data-srcset'))
        else:
            picture_url.append(i.find('a',{'class':'product photo product-item-photo'}).source.get('srcset'))
        
        sales_price.append(i.find('span',{'class':'price'}).text)
        price.append(None)

# Second Layer

In [71]:
product_description = []
material = []
picture_name = []
color = []
counter = 1

for link in product_url:
    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    
    item_color = []
    for color_i in soup.find('ul',{'class':'product-color-select-list'}).find_all('li'):
        item_color.append(color_i.find('div',{'class':"attribute-value"}).text.lower())
    color.append(item_color)
    
    item_material = []
    for material_i in soup.find_all('li',{'class':'composition'}):
        item_material.append(material_i.text.strip().replace('\n',''))
    material.append(item_material)
    
    if soup.find('div',{'class':'product-info-tabs__item'}).find(['div',{'class':'value'}]) != None:
        product_description.append(soup.find('div',{'class':'product-info-tabs__item'}).find(['div',{'class':'value'}]).text.strip())
    else:
        product_description.append(None)
    picture_name.append('kith-women_' + str(counter) + '.jpg') # change variable
    counter += 1

In [72]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Maison Kitsune,Women's Clothing,Tee-shirt flower fox patch,Cotton t-shirt with Flower Fox patch. Rib-knit...,None,$110.00,"[black, light khaki, navy, white]","[0, , ]",https://maisonkitsune.com/us/tee-shirt-flower-...,https://media.maisonkitsune.com/media/catalog/...,2020-12-06,kith-women_1.jpg
1,Maison Kitsune,Women's Clothing,Tee-shirt flower fox patch,Cotton t-shirt with Flower Fox patch. Rib-knit...,None,$110.00,"[white, black, light khaki, navy]","[0, , ]",https://maisonkitsune.com/us/tee-shirt-flower-...,https://media.maisonkitsune.com/media/catalog/...,2020-12-06,kith-women_2.jpg
2,Maison Kitsune,Women's Clothing,Velvet fox head patch oversized r-neck pullover,Wool sweater with velvet Fox patch. Ribbing at...,None,$455.00,[black],"[0, , ]",https://maisonkitsune.com/us/velvet-fox-head-p...,https://media.maisonkitsune.com/media/catalog/...,2020-12-06,kith-women_3.jpg
3,Maison Kitsune,Women's Clothing,Polo embroidery fox,Iconic polo in cotton pique with Navy Fox Head...,None,$165.00,"[grey melange, navy]","[0, , ]",https://maisonkitsune.com/us/polo-embroidery-f...,https://media.maisonkitsune.com/media/catalog/...,2020-12-06,kith-women_4.jpg
4,Maison Kitsune,Women's Clothing,Polo embroidery fox,Iconic polo in cotton pique with Navy Fox Head...,None,$165.00,"[navy, grey melange]","[0, , ]",https://maisonkitsune.com/us/polo-embroidery-f...,https://media.maisonkitsune.com/media/catalog/...,2020-12-06,kith-women_5.jpg


In [73]:
second_pass.to_csv("maison-kitsune-women.csv", index=False, encoding = 'utf-8-sig')

# Download Image

In [74]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/maison-kitsune-women.csv', encoding='utf-8-sig')

os.mkdir('maison-kitsune-women')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('maison-kitsune-women/'+ "maison-kitsune-women_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1